In [1]:
# trying to "emulate" (;)) the composer trade logics    

The Not Boring: Rising Rates with Vol Switch

21.9% annualized return, 23% max drawdown, 1.12 sharpe ratio

Risk Parity, but make it Fed-friendly. This symphony seeks to navigate a rising interest rate environment by including financials and avoiding equity and bond market volatility. The symphony is Risk On if treasuries and the Nasdaq 100 are stable; otherwise, it adds Risk-Off assets, including gold and the US dollar. The Risk-On assets are an inverse-volatility-weighted blend of 3x levered ETFs, representing large-cap equities, financials, and long-dated treasuries. This symphony was developed by Composer. By choosing to trade this strategy, you understand that the strategy employs ETFs, and that you understand the associated risks of trading, holding, and costs associated with ETFs, and also leveraged ETFs. Before investing in ANY strategy containing an ETF, please review the relevant ETF prospectus.

In [ ]:
# if 10d max drawdown of QQQ is greater than 6%
    # "risk off mode"
        # inverse volatility weight 45d = Assets that are more volatile receive lower weights
        # GLD, TMF, FAS, TQQQ, UUP = Gold, Treasury bond, financial bull, 3x qqq, usd bullish
# else 
    # if 10d max drawdown of TMF is greather than 7%
        # "risk off mode"
            # inverse volatility weight 45d = Assets that are more volatile receive lower weights
            # GLD, TMF, FAS, TQQQ, UUP = Gold, Treasury bond, financial bull, 3x qqq, usd bullish
    # else
        # "risk on mode"
        # inverse volatility weight 45d = Assets that are more volatile receive lower weights
        # TMF, FAS, TQQQ (treasury bull 3x, financial bull 3x, 3x qqq)

In [10]:
from basebot22.basebot import BaseBot
from datetime import datetime, timedelta

bot = BaseBot("testbot")
STOCKS = ["GLD", "TMF", "FAS", "TQQQ", "UUP", "QQQ"]


In [48]:

currentMode = "none"

def get10DayMaxDrawdown(ticker):
    qqq = bot.getData(ticker, start_date = datetime.now() - timedelta(days=100))
    # calculate the max percentage drawdown of the last 45 days
    rollMax = qqq['adj_close'].rolling(10, min_periods=1).max()
    dailyDrawdown = qqq['adj_close'] / rollMax - 1.0

    maxDailyDrawdown = dailyDrawdown.rolling(10, min_periods=1).min()

    crntMaxDailyDrawdown = maxDailyDrawdown[-1]

    # print("crnt max daily drawdown qqq: " + str(crntMaxDailyDrawdown))
    return crntMaxDailyDrawdown



In [31]:
import pandas as pd
def calculateCurrent45dVolatility(df: pd.DataFrame):
    df["daily_returns"] = df["adj_close"].pct_change()
    # 45 bc we need 45 d volatility
    df["daily_volatility"] = df["daily_returns"].rolling(45).std()
    return df["daily_volatility"][-1]

In [44]:
def inverseVolatilityWeights(tickers: list):
    crntVolatilities = dict()
    for ticker in tickers:
        df = bot.getData(ticker, start_date = datetime.now() - timedelta(days=100))
        crntVolatilities[ticker] = calculateCurrent45dVolatility(df)
    # create weights according to inverse volatilities (lower volatilities get higher weights)
    print("volatilities: " + str(crntVolatilities))
    for ticker, volatility in crntVolatilities.items():
        crntVolatilities[ticker] = 1 / volatility
    weights = dict()
    for ticker, volatility in crntVolatilities.items():
        weights[ticker] = crntVolatilities[ticker] / sum(crntVolatilities.values())
    print("weights: " + str(weights))
    return weights

In [45]:
# weights = inverseVolatilityWeights(["GLD", "TMF", "FAS", "TQQQ", "UUP"])
# weights

volatilities: {'GLD': 0.010789899006536223, 'TMF': 0.04174950594038894, 'FAS': 0.039695852529190646, 'TQQQ': 0.0593806897897067, 'UUP': 0.007318079356015833}
weights: {'GLD': 0.31383556629146425, 'TMF': 0.08110884161788613, 'FAS': 0.08530498400189027, 'TQQQ': 0.0570261827024278, 'UUP': 0.46272442538633174}


{'GLD': 0.31383556629146425,
 'TMF': 0.08110884161788613,
 'FAS': 0.08530498400189027,
 'TQQQ': 0.0570261827024278,
 'UUP': 0.46272442538633174}

In [50]:
def buyAccordingToWeights(weights: dict):
    portfolio = bot.getPortfolio()
    usd = portfolio["USD"]
    print("portfolio: ", portfolio)
    if len(portfolio) > 1:
        # sell all stocks
        print("selling all open stocks")
        for ticker, amount in portfolio.items():
            if ticker != "USD":
                bot.sell(ticker) # all
        # reset
        portfolio = bot.getPortfolio()
        usd = portfolio["USD"]
    # buy according to weights
    for ticker, weight in weights.items():
        print(f'buying {weight*usd}$ of {ticker}')
        bot.buy(ticker, amount = weight*usd, amountInUSD=True)
    

In [51]:
# trade logic
qqqCrntMaxDailyDrawdown = get10DayMaxDrawdown("QQQ")
if qqqCrntMaxDailyDrawdown < -0.06: # question: how is the "grater than" meant? 
    print("risk off mode")
    if currentMode != "risk off":
        currentMode = "risk off"
        print("switching mode to risk off")
        # get 45d inverse volatility weights
        weights = inverseVolatilityWeights(["GLD", "TMF", "FAS", "TQQQ", "UUP"])
        print(weights)
        buyAccordingToWeights(weights)
else:
    # if 10d max drawdown of TMF is greather than 7%
    # "risk off mode"
    tmfCrntMaxDailyDrawdown = get10DayMaxDrawdown("TMF")
    if tmfCrntMaxDailyDrawdown < -0.07:
        print("risk off mode")
        if currentMode != "risk off":
            currentMode = "risk off"
            print("switching mode to risk off")
            # get 45d inverse volatility weights
            weights = inverseVolatilityWeights(["GLD", "TMF", "FAS", "TQQQ", "UUP"])
            print(weights)
            buyAccordingToWeights(weights)
    else:
        # "risk on mode"
        # inverse volatility weight 45d = Assets that are more volatile receive lower weights
        # TMF, FAS, TQQQ (treasury bull 3x, financial bull 3x, 3x qqq)
        print("risk off on")
        if currentMode != "risk on":
            currentMode = "risk on"
            print("switching mode to risk on")
            # get 45d inverse volatility weights
            weights = inverseVolatilityWeights(["TMF", "FAS", "TQQQ"])
            print(weights)
            buyAccordingToWeights(weights)
    

risk off mode


In [46]:
print(weights)

{'GLD': 0.31383556629146425, 'TMF': 0.08110884161788613, 'FAS': 0.08530498400189027, 'TQQQ': 0.0570261827024278, 'UUP': 0.46272442538633174}
